In [24]:
import pandas as pd
import pickle
import dateparser
from dateutil.parser import parse

In [4]:
with open('2018-04-1211-04-33.411746.pickle', 'rb') as handle:
    master_dict = pickle.load(handle)

In [32]:
with open('2018-04-12-11-04-33.411815_df.pickle', 'rb') as handle:
    df = pickle.load(handle)
    df['reported_at'] = ''
    df['sighted_at'] = ''
    df = df.drop('Id', 1)

In [33]:
df.head(2)

,Captions,Class,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,URL,id,text,location,date,lat,lon,named_entities,DATE,reported_at,sighted_at
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DEFE-24-1922_29,"K. NEARBY OBJECTS\n(Telephone lines, high volt...","RAF, Turnhouse Road, Dolphington Cottages, Ing...",,55.9516,-3.34694,"[(u'TITLE', u'river'), (u'CAUSE_OF_DEATH', u'f...",MAY,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DEFE-24-1922_15,REPORT OF AN UNIDENTIFIED FLYING OBJECT\n\n \n...,"Time, Telegraph Road, Woodchurch, Wirral, Nort...",,53.3287,-3.10015,"[(u'ORGANIZATION', u'Time'), (u'NUMBER', u'27,...",,,


### Go through each row and assign reported_at and sighted_at

In [ ]:
for index,row in df.iterrows():
#     print index
    row_id = row['id']
    ners = master_dict[row_id]
    reported_at = ''
    sighted_at = ''
    cnt = 0
    for nr in ners:
        if nr[0] == 'DATE':
            try:
                if dateparser.parse(nr[1]):
                    cnt += 1

                    if cnt == 2:
                        second_date = nr[1]
                        first_dt = dateparser.parse(first_date)
                        second_dt = dateparser.parse(second_date)
                        if first_dt < second_dt:
                            sighted_at = first_date
                            reported_at = second_date
                        else:
                            sighted_at = second_date
                            reported_at = first_date
                        break
                    else:
                        first_date = nr[1]
                        sighted_at = nr[1]
            except:
                continue
    df.loc[index, 'reported_at'] = reported_at
    df.loc[index, 'sighted_at'] = sighted_at

In [45]:
df.text.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)

In [46]:
df.to_csv('PDFs_reported_sighted_at.csv', index=False, encoding='utf-8')

In [47]:
with open('PDFs_reported_sighted_at.pickle', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)